# **Sending e-mail using Python and Azure Graph API**

In [ ]:
import pandas as pd
import requests
import json
import io
import os
import msal
import base64
from datetime import datetime
import logging
import time

# Set up basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

**SPN configuration**

In [ ]:
# Service Principal Configuration - Replace these values with your actual service principal details
# IMPORTANT: For production, store these values securely (e.g., Azure Key Vault) rather than hardcoding them

# Your Azure AD tenant ID
TENANT_ID = ""  # GUID

# Your registered application (service principal) details
CLIENT_ID = ""     # Application (client) ID
CLIENT_SECRET = ""  # Client secret value

# The sender email address for the service principal
# This must be a valid mailbox that the service principal has permission to send from
SENDER_EMAIL = ""  # Must be a valid mailbox the service principal can access

**Generate Sample data**

In [ ]:
# Generate sample pharmacy data
today = datetime.now().strftime('%Y-%m-%d')

data = {
    'Pharmacy': ['Pharm 1', 'Pharm 2', 'Pharm 3', 'Pharm 4', 'Pharm 5'],
    'Location': ['Amsterdam', 'Rotterdam', 'Utrecht', 'Den Haag', 'Eindhoven'],
    'Total Prescriptions': [120, 95, 110, 88, 105],
    'Revenue': [4500.75, 3850.25, 4210.50, 3200.80, 4050.60],
    'Date': [today, today, today, today, today]
}

# Create DataFrame
df = pd.DataFrame(data)
display(df)

**Create CSV file**

In [ ]:
# Generate timestamp for the filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f"pharm_report_{timestamp}.csv"

# Set the Lakehouse folder path - this is the native Fabric path
lakehouse_folder = '/lakehouse/default/Files/reports/'
lakehouse_csv_path = lakehouse_folder + csv_filename

# Create a buffer to hold CSV content for email attachment
csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=False)
csv_content = csv_buffer.getvalue()

try:
    # Create the directory if it doesn't exist (for Fabric)
    os.makedirs(lakehouse_folder, exist_ok=True)
    
    # Save the DataFrame directly to the Lakehouse folder
    df.to_csv(lakehouse_csv_path, index=False)
    logging.info(f'CSV saved directly to Fabric Lakehouse: {lakehouse_csv_path}')
    
    # The file path for later use (in Fabric environment)
    report_path = lakehouse_csv_path
except Exception as e:
    logging.warning(f'Could not save to Lakehouse (normal if not running in Fabric): {e}')
    
    # For local testing when not in Fabric, create a local file
    local_path = os.path.join(os.getcwd(), csv_filename)
    df.to_csv(local_path, index=False)
    logging.info(f"CSV file created locally at: {local_path} (fallback for non-Fabric environments)")
    
    # The file path for later use (in non-Fabric environment)
    report_path = local_path

**Send email with CSV attachment**

In [ ]:
#
## Email details - Update with your recipient email address
recipient_email = ""  # Replace with the actual recipient email

subject = f"Pharmacy Report - {today}"

# Create email body with a summary of the report
total_prescriptions = df['Total Prescriptions'].sum()
total_revenue = df['Revenue'].sum()

email_body = f"""<html>
<body>
<h2>Pharmacy Report</h2>
<p>Dear Recipient,</p>
<p>Please find attached the daily pharmacy report for {today}.</p>
<h3>Summary:</h3>
<ul>
  <li>Total pharmacies: {len(df)}</li>
  <li>Total prescriptions: {total_prescriptions}</li>
  <li>Total revenue: €{total_revenue:.2f}</li>
</ul>
<p>The detailed breakdown is available in the attached CSV file.</p>
<p>Best regards,<br>Report Generator</p>
</body>
</html>"""

print(f"Email will be sent to: {recipient_email}")
print(f"Subject: {subject}")

In [ ]:
def get_token_with_service_principal():
    """
    Get an access token for Microsoft Graph API using a service principal.
    
    Returns:
        str: Access token if successful, None if failed
    """
    try:
        # Verify configuration is set
        if TENANT_ID == "your-tenant-id" or CLIENT_ID == "your-client-id" or CLIENT_SECRET == "your-client-secret":
            logging.error("Service principal configuration is not set. Please update the configuration in Section 2.")
            return None
        
        # Configuration for the Microsoft Graph API
        authority = f"https://login.microsoftonline.com/{TENANT_ID}"
        scope = ["https://graph.microsoft.com/.default"]  # Using .default scope with client credentials flow
        
        # Create a confidential client application
        app = msal.ConfidentialClientApplication(
            client_id=CLIENT_ID,
            client_credential=CLIENT_SECRET,
            authority=authority
        )
        
        # Acquire token for the app (client credentials flow)
        logging.info("Acquiring token for Microsoft Graph API using service principal...")
        result = app.acquire_token_for_client(scopes=scope)
        
        if "access_token" in result:
            logging.info("Successfully acquired token using service principal")
            return result["access_token"]
        else:
            logging.error(f"Failed to acquire token: {result.get('error')}")
            logging.error(f"Error description: {result.get('error_description')}")
            return None
            
    except Exception as e:
        logging.error(f"Error in get_token_with_service_principal: {e}")
        return None

In [ ]:
def send_email_with_graph(token, sender_email, recipient_email, subject, body_html, attachment_content, attachment_name):
    """
    Send an email with attachment using Microsoft Graph API.
    
    Args:
        token (str): The access token for Microsoft Graph API
        sender_email (str): Email address to send from (must be accessible by the service principal)
        recipient_email (str): Email address of the recipient
        subject (str): Subject line of the email
        body_html (str): HTML content for the email body
        attachment_content (str): Content of the attachment
        attachment_name (str): Filename for the attachment
    
    Returns:
        bool: True if email sent successfully, False otherwise
    """
    try:
        # Encode the attachment content in base64
        attachment_content_bytes = attachment_content.encode("utf-8")
        attachment_base64 = base64.b64encode(attachment_content_bytes).decode("utf-8")
        
        # Prepare the email message
        email_message = {
            "message": {
                "subject": subject,
                "body": {
                    "contentType": "HTML",
                    "content": body_html
                },
                "toRecipients": [
                    {
                        "emailAddress": {
                            "address": recipient_email
                        }
                    }
                ],
                "attachments": [
                    {
                        "@odata.type": "#microsoft.graph.fileAttachment",
                        "name": attachment_name,
                        "contentType": "text/csv",
                        "contentBytes": attachment_base64
                    }
                ]
            },
            "saveToSentItems": "true"
        }
        
        # When using a service principal, we need to use the users/{userId}/sendMail endpoint
        # instead of /me/sendMail
        graph_endpoint = f"https://graph.microsoft.com/v1.0/users/{sender_email}/sendMail"
        
        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        }
        
        # Implement retry logic with backoff for transient failures
        max_retries = 3
        retry_delay = 2  # seconds
        
        for attempt in range(max_retries):
            try:
                logging.info(f"Sending email from {sender_email} to {recipient_email}, attempt {attempt+1} of {max_retries}")
                response = requests.post(graph_endpoint, headers=headers, data=json.dumps(email_message), timeout=30)
                
                # Check response body for more details
                try:
                    response_json = response.json() if response.text else {"message": "No response body"}
                except:
                    response_json = {"message": "Unable to parse response as JSON"}
                
                if response.status_code == 202:
                    logging.info(f"Email sent successfully to {recipient_email}")
                    return True
                else:
                    logging.warning(f"Attempt {attempt+1}: Failed to send email. Status code: {response.status_code}")
                    logging.warning(f"Response details: {response_json}")
                    
                    # Special handling for common errors
                    if "error" in response_json:
                        if "code" in response_json["error"]:
                            error_code = response_json["error"]["code"]
                            if error_code == "Authorization_RequestDenied":
                                logging.error("The service principal doesn't have permission to send mail as this user.")
                                return False
                            elif error_code == "ResourceNotFound":
                                logging.error(f"The specified sender email '{sender_email}' could not be found.")
                                return False
                    
                    if attempt < max_retries - 1:
                        logging.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                        retry_delay *= 2  # Exponential backoff
                    else:
                        logging.error(f"Failed to send email after {max_retries} attempts.")
            except requests.exceptions.RequestException as e:
                if attempt < max_retries - 1:
                    logging.warning(f"Request exception: {e}. Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                    retry_delay *= 2  # Exponential backoff
                else:
                    logging.error(f"Request exception after {max_retries} attempts: {e}")
        
        return False
    except Exception as e:
        logging.error(f"Error in send_email_with_graph: {e}")
        return False

In [ ]:
def send_report_email():
    """
    Main function to send the report email using Microsoft Graph API with service principal authentication.
    """
    try:
        # Step 1: Get access token using service principal
        logging.info("Starting email sending process...")
        token = get_token_with_service_principal()
        
        if not token:
            logging.error("Failed to obtain access token. Check the service principal configuration and permissions.")
            return False
            
        # Step 2: Send the email with the CSV attachment
        result = send_email_with_graph(
            token=token,
            sender_email=SENDER_EMAIL,
            recipient_email=recipient_email,
            subject=subject,
            body_html=email_body,
            attachment_content=csv_buffer.getvalue(),
            attachment_name=csv_filename
        )
        
        return result
    except Exception as e:
        logging.error(f"Error in send_report_email: {e}")
        return False

In [ ]:
send_report_email()